In [1]:
from itertools import permutations
from mnemonic import Mnemonic
from bip_utils import Bip39SeedGenerator, Bip44, Bip44Coins, Bip44Changes
import requests

# Membaca daftar kata dari file
def load_wordlist(file_path):
    with open(file_path, 'r') as file:
        words = file.read().splitlines()
    return words

# Fungsi untuk memeriksa saldo wallet menggunakan API publik (blockchain.com dalam contoh ini)
def check_balance(address):
    url = f"https://blockchain.info/q/addressbalance/{address}"
    response = requests.get(url)
    if response.status_code == 200:
        balance = int(response.text) / 1e8  # Konversi satoshi ke BTC
        return balance
    return None

# Fungsi untuk mendapatkan alamat Bitcoin dari seed phrase
def get_btc_address(seed_phrase):
    seed_bytes = Bip39SeedGenerator(seed_phrase).Generate()
    bip44_mst_ctx = Bip44.FromSeed(seed_bytes, Bip44Coins.BITCOIN)
    bip44_acc_ctx = bip44_mst_ctx.Purpose().Coin().Account(0)
    bip44_chg_ctx = bip44_acc_ctx.Change(Bip44Changes.CHAIN_EXT)
    bip44_addr_ctx = bip44_chg_ctx.AddressIndex(0)
    return bip44_addr_ctx.PublicKey().ToAddress()

# Memuat daftar kata
words = load_wordlist('seedphrase_words.txt')

# Loop untuk mencoba setiap kombinasi seed phrase
mnemo = Mnemonic('english')
for phrase_tuple in permutations(words, 12):  # Menggunakan 12 kata dari daftar kata
    seed_phrase = ' '.join(phrase_tuple)
    if mnemo.check(seed_phrase):
        address = get_btc_address(seed_phrase)
        balance = check_balance(address)
        if balance is not None and balance > 0:
            print(f"Found non-zero balance! Seed Phrase: {seed_phrase}, Address: {address}, Balance: {balance} BTC")
            break
        else:
            print(f"Checked address {address} with seed phrase {seed_phrase}, balance: {balance} BTC")

print("Completed checking seed phrases.")

Checked address 1LcStvg9StowHkWShpBq2YckG4xSycKXFT with seed phrase abandon ability able about above absent absorb abstract absurd abuse access actress, balance: None BTC
Checked address 17oGAa3eecHkZPRbJMU5sxQ1mQEBDZzEvE with seed phrase abandon ability able about above absent absorb abstract absurd abuse access aisle, balance: None BTC
Checked address 15CFw2jLLaqyiCDzgDmmqMtrWqyZ36pDNM with seed phrase abandon ability able about above absent absorb abstract absurd abuse access alert, balance: None BTC
Checked address 1BnVXexLitdXwfgxAr57pVvVUWZaB5HGhR with seed phrase abandon ability able about above absent absorb abstract absurd abuse access analyst, balance: None BTC
Checked address 146yLokB85GzP3NVp1n4M8TfucwQznVLem with seed phrase abandon ability able about above absent absorb abstract absurd abuse access any, balance: None BTC
Checked address 1EV5TJDaBgcYjXbZynsHge2AE2mszd7m2g with seed phrase abandon ability able about above absent absorb abstract absurd abuse access arrive, b

KeyboardInterrupt: 

In [9]:
import csv
import random
import time
import psutil
from itertools import permutations
from mnemonic import Mnemonic
from bip_utils import Bip39SeedGenerator, Bip44, Bip44Coins, Bip44Changes
import requests
import os

# Membaca daftar kata dari file
def load_wordlist(file_path):
    with open(file_path, 'r') as file:
        words = file.read().splitlines()
    return words

# Mengacak seed phrase secara acak
def generate_random_seed_phrase(words):
    return ' '.join(random.sample(words, 12))

# Fungsi untuk memeriksa saldo wallet menggunakan API publik (blockchain.com dalam contoh ini)
def check_balance(address):
    url = f"https://blockchain.info/q/addressbalance/{address}"
    response = requests.get(url)
    if response.status_code == 200:
        balance = int(response.text) / 1e8  # Konversi satoshi ke BTC
        return balance
    return None

# Fungsi untuk mendapatkan alamat Bitcoin dari seed phrase
def get_btc_address(seed_phrase):
    seed_bytes = Bip39SeedGenerator(seed_phrase).Generate()
    bip44_mst_ctx = Bip44.FromSeed(seed_bytes, Bip44Coins.BITCOIN)
    bip44_acc_ctx = bip44_mst_ctx.Purpose().Coin().Account(0)
    bip44_chg_ctx = bip44_acc_ctx.Change(Bip44Changes.CHAIN_EXT)
    bip44_addr_ctx = bip44_chg_ctx.AddressIndex(0)
    return bip44_addr_ctx.PublicKey().ToAddress()

# Memuat daftar kata
words = load_wordlist('seedphrase_words.txt')

# Inisialisasi Mnemonic
mnemo = Mnemonic('english')

# Nama file CSV untuk mencatat hasil
csv_filename = 'seedphrase_attempts.csv'

# Membuat file CSV jika belum ada dan menulis header
def initialize_csv(filename):
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ['time', 'address', 'seed_phrase', 'balance']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

# Menambahkan entri ke file CSV
def log_attempt(filename, address, seed_phrase, balance):
    with open(filename, 'a', newline='') as csvfile:
        fieldnames = ['time', 'address', 'seed_phrase', 'balance']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow({'time': time.strftime("%Y-%m-%d %H:%M:%S"), 'address': address, 'seed_phrase': seed_phrase, 'balance': balance})

# Membaca seed phrase yang sudah dicoba dari file CSV
def read_attempts(filename):
    tried_phrases = set()
    if not os.path.exists(filename):
        return tried_phrases
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            tried_phrases.add(row['seed_phrase'])
    return tried_phrases

# Menghitung konsumsi daya CPU/GPU dan estimasi listrik
def get_power_usage():
    cpu_usage = psutil.cpu_percent(interval=1)
    num_cores = psutil.cpu_count(logical=False)
    power_per_core_watts = 10  # asumsi rata-rata daya per core CPU dalam watt
    total_power_usage_watts = cpu_usage / 100.0 * num_cores * power_per_core_watts
    total_power_usage_kwh = total_power_usage_watts / 1000.0 / 3600.0  # konversi ke kWh
    return total_power_usage_watts, total_power_usage_kwh

# Inisialisasi file CSV jika belum ada
if not os.path.exists(csv_filename):
    initialize_csv(csv_filename)

# Membaca seed phrase yang sudah dicoba
tried_phrases = read_attempts(csv_filename)

# Hitung waktu mulai
start_time = time.time()

# Loop untuk mencoba seed phrase acak
total_attempts = 0
while True:
    seed_phrase = generate_random_seed_phrase(words)
    if seed_phrase in tried_phrases:
        continue
    if mnemo.check(seed_phrase):
        address = get_btc_address(seed_phrase)
        balance = check_balance(address)
        log_attempt(csv_filename, address, seed_phrase, balance)
        tried_phrases.add(seed_phrase)
        total_attempts += 1
        if balance is not None and balance > 0:
            print(f"Found non-zero balance! Seed Phrase: {seed_phrase}, Address: {address}, Balance: {balance} BTC")
            break
        end_time = time.time()
        elapsed_time = end_time - start_time
        if elapsed_time > 0:
            addresses_per_second = total_attempts / elapsed_time
            power_usage_watts, power_usage_kwh = get_power_usage()
            print(f"Addresses checked per second: {addresses_per_second:.2f}, Power Usage: {power_usage_watts:.2f} W, Power Usage (kWh): {power_usage_kwh:.6f}")

print("Completed checking seed phrases.")

Addresses checked per second: 8.88, Power Usage: 3.04 W, Power Usage (kWh): 0.000001
Addresses checked per second: 1.63, Power Usage: 2.56 W, Power Usage (kWh): 0.000001
Addresses checked per second: 1.27, Power Usage: 2.40 W, Power Usage (kWh): 0.000001
Addresses checked per second: 1.14, Power Usage: 1.92 W, Power Usage (kWh): 0.000001
Addresses checked per second: 1.08, Power Usage: 2.72 W, Power Usage (kWh): 0.000001
Addresses checked per second: 1.05, Power Usage: 3.36 W, Power Usage (kWh): 0.000001
Addresses checked per second: 1.02, Power Usage: 2.56 W, Power Usage (kWh): 0.000001
Addresses checked per second: 1.00, Power Usage: 1.44 W, Power Usage (kWh): 0.000000
Addresses checked per second: 0.99, Power Usage: 1.60 W, Power Usage (kWh): 0.000000
Addresses checked per second: 0.98, Power Usage: 1.76 W, Power Usage (kWh): 0.000000
Addresses checked per second: 0.97, Power Usage: 3.36 W, Power Usage (kWh): 0.000001
Addresses checked per second: 0.96, Power Usage: 2.08 W, Power Us

KeyboardInterrupt: 

In [1]:
import csv
import random
import time
import psutil
import os
import requests
import platform
from itertools import permutations
from mnemonic import Mnemonic
from bip_utils import Bip39SeedGenerator, Bip44, Bip44Coins, Bip44Changes
from concurrent.futures import ThreadPoolExecutor, as_completed
import sys

# Membaca daftar kata dari file
def load_wordlist(file_path):
    with open(file_path, 'r') as file:
        words = file.read().splitlines()
    return words

# Mengacak seed phrase secara acak
def generate_random_seed_phrase(words):
    return ' '.join(random.sample(words, 12))

# Fungsi untuk memeriksa saldo wallet menggunakan API publik (blockchain.com dalam contoh ini)
def check_balance(address):
    url = f"https://blockchain.info/q/addressbalance/{address}"
    response = requests.get(url)
    if response.status_code == 200:
        balance = int(response.text) / 1e8  # Konversi satoshi ke BTC
        return balance
    return None

# Fungsi untuk mendapatkan alamat Bitcoin dari seed phrase
def get_btc_address(seed_phrase):
    seed_bytes = Bip39SeedGenerator(seed_phrase).Generate()
    bip44_mst_ctx = Bip44.FromSeed(seed_bytes, Bip44Coins.BITCOIN)
    bip44_acc_ctx = bip44_mst_ctx.Purpose().Coin().Account(0)
    bip44_chg_ctx = bip44_acc_ctx.Change(Bip44Changes.CHAIN_EXT)
    bip44_addr_ctx = bip44_chg_ctx.AddressIndex(0)
    return bip44_addr_ctx.PublicKey().ToAddress()

# Memuat daftar kata
words = load_wordlist('seedphrase_words.txt')

# Inisialisasi Mnemonic
mnemo = Mnemonic('english')

# Nama file CSV untuk mencatat hasil
csv_filename = 'seedphrase_attempts.csv'

# Membuat file CSV jika belum ada dan menulis header
def initialize_csv(filename):
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ['time', 'address', 'seed_phrase', 'balance']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

# Menambahkan entri ke file CSV
def log_attempt(filename, address, seed_phrase, balance):
    with open(filename, 'a', newline='') as csvfile:
        fieldnames = ['time', 'address', 'seed_phrase', 'balance']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow({'time': time.strftime("%Y-%m-%d %H:%M:%S"), 'address': address, 'seed_phrase': seed_phrase, 'balance': balance})

# Membaca seed phrase yang sudah dicoba dari file CSV
def read_attempts(filename):
    tried_phrases = set()
    if not os.path.exists(filename):
        return tried_phrases
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            tried_phrases.add(row['seed_phrase'])
    return tried_phrases

# Menghitung konsumsi daya CPU dan estimasi listrik
def get_power_usage():
    cpu_usage = psutil.cpu_percent(interval=1)
    num_cores = psutil.cpu_count(logical=False)
    cpu_freq = psutil.cpu_freq().current  # Get current CPU frequency in MHz
    power_per_core_watts = 10  # asumsi rata-rata daya per core CPU dalam watt
    total_power_usage_watts = cpu_usage / 100.0 * num_cores * power_per_core_watts
    total_power_usage_kwh = total_power_usage_watts / 1000.0 / 3600.0  # konversi ke kWh
    return total_power_usage_watts, total_power_usage_kwh, num_cores, cpu_freq

# Menghitung biaya listrik
def calculate_electricity_cost(total_power_usage_kwh):
    rate_per_kwh = 1699.53  # Rp per kWh
    cost_per_day = total_power_usage_kwh * 24 * rate_per_kwh
    cost_per_month = cost_per_day * 30
    return cost_per_day, cost_per_month

# Menghitung estimasi waktu yang dibutuhkan untuk mencoba semua kemungkinan seed phrase
def estimate_total_time(addresses_per_second, total_combinations):
    if addresses_per_second == 0:
        return float('inf')
    total_seconds = total_combinations / addresses_per_second
    years = total_seconds / (60 * 60 * 24 * 365.25)  # Konversi ke tahun
    return years

# Inisialisasi file CSV jika belum ada
if not os.path.exists(csv_filename):
    initialize_csv(csv_filename)

# Membaca seed phrase yang sudah dicoba
tried_phrases = read_attempts(csv_filename)

# Dapatkan spesifikasi CPU
cpu_brand = platform.processor()
cpu_freq_info = psutil.cpu_freq()

# Hitung waktu mulai
start_time = time.time()

# Loop untuk mencoba seed phrase acak
total_attempts = 0
total_combinations = len(words) ** 12  # Total kemungkinan kombinasi

def check_seed_phrase():
    global total_attempts, start_time
    while True:
        seed_phrase = generate_random_seed_phrase(words)
        if seed_phrase in tried_phrases:
            continue
        if mnemo.check(seed_phrase):
            address = get_btc_address(seed_phrase)
            balance = check_balance(address)
            log_attempt(csv_filename, address, seed_phrase, balance)
            tried_phrases.add(seed_phrase)
            total_attempts += 1
            end_time = time.time()
            elapsed_time = end_time - start_time
            addresses_per_second = total_attempts / elapsed_time
            power_usage_watts, power_usage_kwh, num_cores, cpu_freq = get_power_usage()
            cost_per_day, cost_per_month = calculate_electricity_cost(power_usage_kwh)
            total_years = estimate_total_time(addresses_per_second, total_combinations)
            info = (f"\rAddresses checked per second: {addresses_per_second:.2f}\n"
                    f"Power Usage: {power_usage_watts:.2f} W\n"
                    f"Power Usage (kWh): {power_usage_kwh:.6f}\n"
                    f"CPU Cores: {num_cores}\n"
                    f"CPU Frequency: {cpu_freq_info.current:.2f} MHz\n"
                    f"CPU: {cpu_brand}\n"
                    f"Estimated Electricity Cost per Day: Rp {cost_per_day:.2f}\n"
                    f"Estimated Electricity Cost per Month: Rp {cost_per_month:.2f}\n"
                    f"Estimated Time to Check All Combinations: {total_years:.2f} years")
            sys.stdout.write(info)
            sys.stdout.flush()
            if balance is not None and balance > 0:
                print(f"\nFound non-zero balance! Seed Phrase: {seed_phrase}, Address: {address}, Balance: {balance} BTC")
                break

# Menggunakan ThreadPoolExecutor untuk memaksimalkan penggunaan CPU
with ThreadPoolExecutor(max_workers=psutil.cpu_count(logical=True)) as executor:
    futures = [executor.submit(check_seed_phrase) for _ in range(psutil.cpu_count(logical=True))]
    for future in as_completed(futures):
        future.result()

print("\nCompleted checking seed phrases.")

Addresses checked per second: 1.30
Power Usage: 1.28 W
Power Usage (kWh): 0.000000
CPU Cores: 16
CPU Frequency: 3200.00 MHz
CPU: Intel64 Family 6 Model 151 Stepping 2, GenuineIntel
Estimated Electricity Cost per Day: Rp 0.01
Estimated Electricity Cost per Month: Rp 0.44
Addresses checked per second: 2.59ations: 132926985322403713478476041289728.00 years
Power Usage: 1.28 W
Power Usage (kWh): 0.000000
CPU Cores: 16
CPU Frequency: 3200.00 MHz
CPU: Intel64 Family 6 Model 151 Stepping 2, GenuineIntel
Estimated Electricity Cost per Day: Rp 0.01
Estimated Electricity Cost per Month: Rp 0.44
Addresses checked per second: 3.78ations: 66549790722597573391654146015232.00 years
Power Usage: 1.28 W
Power Usage (kWh): 0.000000
CPU Cores: 16
CPU Frequency: 3200.00 MHz
CPU: Intel64 Family 6 Model 151 Stepping 2, GenuineIntel
Estimated Electricity Cost per Day: Rp 0.01
Estimated Electricity Cost per Month: Rp 0.44
Addresses checked per second: 5.03ations: 45698731190554528895415359635456.00 years
Powe

In [11]:
import csv
import itertools
import time
import psutil
import os
import requests
import platform
from mnemonic import Mnemonic
from bip_utils import Bip39SeedGenerator, Bip44, Bip44Coins, Bip44Changes
import sys
from concurrent.futures import ThreadPoolExecutor, as_completed

# Membaca daftar kata dari file
def load_wordlist(file_path):
    with open(file_path, 'r') as file:
        words = file.read().splitlines()
    return words

# Fungsi untuk memeriksa saldo wallet menggunakan API publik (blockchain.com dalam contoh ini)
def check_balance(address):
    url = f"https://blockchain.info/q/addressbalance/{address}"
    response = requests.get(url)
    if response.status_code == 200:
        balance = int(response.text) / 1e8  # Konversi satoshi ke BTC
        return balance
    return None

# Fungsi untuk mendapatkan alamat Bitcoin dari seed phrase
def get_btc_address(seed_phrase):
    seed_bytes = Bip39SeedGenerator(seed_phrase).Generate()
    bip44_mst_ctx = Bip44.FromSeed(seed_bytes, Bip44Coins.BITCOIN)
    bip44_acc_ctx = bip44_mst_ctx.Purpose().Coin().Account(0)
    bip44_chg_ctx = bip44_acc_ctx.Change(Bip44Changes.CHAIN_EXT)
    bip44_addr_ctx = bip44_chg_ctx.AddressIndex(0)
    return bip44_addr_ctx.PublicKey().ToAddress()

# Memuat daftar kata
words = load_wordlist('seedphrase_words.txt')

# Inisialisasi Mnemonic
mnemo = Mnemonic('english')

# Nama file CSV untuk mencatat hasil dan file indeks
csv_filename = 'seedphrase_attempts.csv'
index_filename = 'last_index.txt'

# Membuat file CSV jika belum ada dan menulis header
def initialize_csv(filename):
    if not os.path.exists(filename):
        with open(filename, 'w', newline='') as csvfile:
            fieldnames = ['time', 'address', 'seed_phrase', 'balance']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

# Menambahkan entri ke file CSV
def log_attempt(filename, address, seed_phrase, balance):
    with open(filename, 'a', newline='') as csvfile:
        fieldnames = ['time', 'address', 'seed_phrase', 'balance']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow({'time': time.strftime("%Y-%m-%d %H:%M:%S"), 'address': address, 'seed_phrase': seed_phrase, 'balance': balance})

# Membaca indeks terakhir yang dicoba dari file
def read_last_index(filename):
    if not os.path.exists(filename):
        return 0
    with open(filename, 'r') as file:
        return int(file.read().strip())

# Menyimpan indeks terakhir yang dicoba ke file
def save_last_index(filename, index):
    with open(filename, 'w') as file:
        file.write(str(index))

# Menghitung konsumsi daya CPU dan estimasi listrik
def get_power_usage():
    cpu_usage = psutil.cpu_percent(interval=1)
    num_cores = psutil.cpu_count(logical=False)
    cpu_freq = psutil.cpu_freq().current  # Get current CPU frequency in MHz
    power_per_core_watts = 10  # asumsi rata-rata daya per core CPU dalam watt
    total_power_usage_watts = cpu_usage / 100.0 * num_cores * power_per_core_watts
    total_power_usage_kwh = total_power_usage_watts / 1000.0 / 3600.0  # konversi ke kWh
    return total_power_usage_watts, total_power_usage_kwh, num_cores, cpu_freq

# Menghitung biaya listrik
def calculate_electricity_cost(total_power_usage_kwh):
    rate_per_kwh = 1699.53  # Rp per kWh
    cost_per_day = total_power_usage_kwh * 24 * rate_per_kwh
    cost_per_month = cost_per_day * 30
    return cost_per_day, cost_per_month

# Menghitung estimasi waktu yang dibutuhkan untuk mencoba semua kemungkinan seed phrase
def estimate_total_time(addresses_per_second, total_combinations):
    if addresses_per_second == 0:
        return float('inf')
    total_seconds = total_combinations / addresses_per_second
    years = total_seconds / (60 * 60 * 24 * 365.25)  # Konversi ke tahun
    return years

# Inisialisasi file CSV jika belum ada
initialize_csv(csv_filename)

# Membaca seed phrase yang sudah dicoba
tried_phrases = set()
if os.path.exists(csv_filename):
    with open(csv_filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            tried_phrases.add(row['seed_phrase'])

# Dapatkan spesifikasi CPU
cpu_brand = platform.processor()
cpu_freq_info = psutil.cpu_freq()

# Hitung waktu mulai
start_time = time.time()

# Memuat indeks terakhir yang dicoba
last_index = read_last_index(index_filename)

# Total kemungkinan kombinasi
total_combinations = len(words) ** 12

# Generator untuk seed phrases
def seed_phrase_generator():
    for i, combination in enumerate(itertools.product(words, repeat=12)):
        if i >= last_index:
            yield i, ' '.join(combination)

# Fungsi untuk memeriksa seed phrase
def check_seed_phrase(seed_phrase, index):
    global total_attempts, start_time
    if mnemo.check(seed_phrase):
        address = get_btc_address(seed_phrase)
        balance = check_balance(address)
        log_attempt(csv_filename, address, seed_phrase, balance)
        save_last_index(index_filename, index)
        total_attempts += 1
        end_time = time.time()
        elapsed_time = end_time - start_time
        addresses_per_second = total_attempts / elapsed_time
        power_usage_watts, power_usage_kwh, num_cores, cpu_freq = get_power_usage()
        cost_per_day, cost_per_month = calculate_electricity_cost(power_usage_kwh)
        total_years = estimate_total_time(addresses_per_second, total_combinations)
        info = (f"\rAddresses checked per second: {addresses_per_second:.2f}\n"
                f"Power Usage: {power_usage_watts:.2f} W\n"
                f"Power Usage (kWh): {power_usage_kwh:.6f}\n"
                f"CPU Cores: {num_cores}\n"
                f"CPU Frequency: {cpu_freq_info.current:.2f} MHz\n"
                f"CPU: {cpu_brand}\n"
                f"Estimated Electricity Cost per Day: Rp {cost_per_day:.2f}\n"
                f"Estimated Electricity Cost per Month: Rp {cost_per_month:.2f}\n"
                f"Estimated Time to Check All Combinations: {total_years:.2f} years")
        sys.stdout.write(info)
        sys.stdout.flush()
        if balance is not None and balance > 0:
            print(f"\nFound non-zero balance! Seed Phrase: {seed_phrase}, Address: {address}, Balance: {balance} BTC")
            sys.exit()

# Menggunakan ThreadPoolExecutor untuk memaksimalkan penggunaan CPU
with ThreadPoolExecutor(max_workers=psutil.cpu_count(logical=True)) as executor:
    futures = {executor.submit(check_seed_phrase, seed_phrase, index): index for index, seed_phrase in seed_phrase_generator()}
    for future in as_completed(futures):
        future.result()

print("\nCompleted checking seed phrases.")

Addresses checked per second: 1.61
Power Usage: 2.08 W
Power Usage (kWh): 0.000001
CPU Cores: 16
CPU Frequency: 3200.00 MHz
CPU: Intel64 Family 6 Model 151 Stepping 2, GenuineIntel
Estimated Electricity Cost per Day: Rp 0.02
Estimated Electricity Cost per Month: Rp 0.71
Addresses checked per second: 0.81ations: 106955382191262773964614571917312.00 years
Power Usage: 2.08 W
Power Usage (kWh): 0.000001
CPU Cores: 16
CPU Frequency: 3200.00 MHz
CPU: Intel64 Family 6 Model 151 Stepping 2, GenuineIntel
Estimated Electricity Cost per Day: Rp 0.02
Estimated Electricity Cost per Month: Rp 0.71
Addresses checked per second: 1.95ations: 213910764382525547929229143834624.00 years
Power Usage: 3.84 W
Power Usage (kWh): 0.000001
CPU Cores: 16
CPU Frequency: 3200.00 MHz
CPU: Intel64 Family 6 Model 151 Stepping 2, GenuineIntel
Estimated Electricity Cost per Day: Rp 0.04
Estimated Electricity Cost per Month: Rp 1.31
Addresses checked per second: 2.58ations: 88286851911667918259645273079808.00 years
Pow

In [1]:
import platform
import random
from itertools import permutations
from mnemonic import Mnemonic
from bip_utils import Bip39SeedGenerator, Bip44, Bip44Coins, Bip44Changes
import requests
import os
import time
import psutil
import sys

# Membaca daftar kata dari file
def load_wordlist(file_path):
    with open(file_path, 'r') as file:
        words = file.read().splitlines()
    return words

# Fungsi untuk memeriksa saldo wallet menggunakan API publik (blockchain.com dalam contoh ini)
def check_balance(address):
    url = f"https://blockchain.info/q/addressbalance/{address}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        balance = int(response.text) / 1e8  # Konversi satoshi ke BTC
        return balance
    except requests.RequestException as e:
        print(f"Error checking balance for address {address}: {e}")
        return None

# Fungsi untuk mendapatkan alamat Bitcoin dari seed phrase
def get_btc_address(seed_phrase):
    seed_bytes = Bip39SeedGenerator(seed_phrase).Generate()
    bip44_mst_ctx = Bip44.FromSeed(seed_bytes, Bip44Coins.BITCOIN)
    bip44_acc_ctx = bip44_mst_ctx.Purpose().Coin().Account(0)
    bip44_chg_ctx = bip44_acc_ctx.Change(Bip44Changes.CHAIN_EXT)
    bip44_addr_ctx = bip44_chg_ctx.AddressIndex(0)
    return bip44_addr_ctx.PublicKey().ToAddress()

# Fungsi untuk memuat seed phrase yang sudah dicoba dari file
def load_checked_phrases(file_path):
    if not os.path.exists(file_path):
        return set()
    with open(file_path, 'r') as file:
        return set(line.strip().split(' | ')[2] for line in file)

# Fungsi untuk mencatat seed phrase yang sudah dicoba ke file
def save_checked_phrase(file_path, seed_phrase, address, balance):
    with open(file_path, 'a') as file:
        file.write(f"{time.strftime('%Y-%m-%d %H:%M:%S')} | {address} | {seed_phrase} | {balance} BTC\n")

# Menghitung konsumsi daya CPU dan estimasi listrik
def get_power_usage():
    cpu_usage = psutil.cpu_percent(interval=1)
    num_cores = psutil.cpu_count(logical=False)
    cpu_freq = psutil.cpu_freq().current  # Get current CPU frequency in MHz
    power_per_core_watts = 10  # asumsi rata-rata daya per core CPU dalam watt
    total_power_usage_watts = cpu_usage / 100.0 * num_cores * power_per_core_watts
    total_power_usage_kwh = total_power_usage_watts / 1000.0 / 3600.0  # konversi ke kWh
    return total_power_usage_watts, total_power_usage_kwh, num_cores, cpu_freq

# Menghitung biaya listrik
def calculate_electricity_cost(total_power_usage_kwh):
    rate_per_kwh = 1699.53  # Rp per kWh
    cost_per_day = total_power_usage_kwh * 24 * rate_per_kwh
    cost_per_month = cost_per_day * 30
    return cost_per_day, cost_per_month

# Menghitung estimasi waktu yang dibutuhkan untuk mencoba semua kemungkinan seed phrase
def estimate_total_time(addresses_per_second, total_combinations):
    if addresses_per_second == 0:
        return float('inf')
    total_seconds = total_combinations / addresses_per_second
    years = total_seconds / (60 * 60 * 24 * 365.25)  # Konversi ke tahun
    return years

# Memuat daftar kata
words = load_wordlist('seedphrase_words.txt')
checked_phrases_file = 'checked_phrases.txt'
checked_phrases = load_checked_phrases(checked_phrases_file)

# Inisialisasi Mnemonic
mnemo = Mnemonic('english')

# Dapatkan spesifikasi CPU
cpu_brand = platform.processor()
cpu_freq_info = psutil.cpu_freq()

# Hitung waktu mulai
start_time = time.time()

# Total kemungkinan kombinasi
total_combinations = len(words) ** 12

# Loop untuk mencoba setiap kombinasi seed phrase
total_attempts = 0
while True:
    seed_phrase = ' '.join(random.sample(words, 12))
    if seed_phrase in checked_phrases:
        continue
    if mnemo.check(seed_phrase):
        address = get_btc_address(seed_phrase)
        balance = check_balance(address)
        if balance is None:
            continue  # Jika pengecekan balance gagal, lanjutkan ke iterasi berikutnya
        save_checked_phrase(checked_phrases_file, seed_phrase, address, balance)
        checked_phrases.add(seed_phrase)
        total_attempts += 1
        end_time = time.time()
        elapsed_time = end_time - start_time
        addresses_per_second = total_attempts / elapsed_time
        power_usage_watts, power_usage_kwh, num_cores, cpu_freq = get_power_usage()
        cost_per_day, cost_per_month = calculate_electricity_cost(power_usage_kwh)
        total_years = estimate_total_time(addresses_per_second, total_combinations)
        balance_status = "0 BTC" if balance == 0 else f"{balance} BTC"
        info = (f"\rAddresses checked per second: {addresses_per_second:.2f}\n"
                f"Total tried: {total_attempts}\n"
                f"Power Usage: {power_usage_watts:.2f} W\n"
                f"Power Usage (kWh): {power_usage_kwh:.6f}\n"
                f"CPU Cores: {num_cores}\n"
                f"CPU Frequency: {cpu_freq_info.current:.2f} MHz\n"
                f"CPU: {cpu_brand}\n"
                f"Estimated Electricity Cost per Day: Rp {cost_per_day:.2f}\n"
                f"Estimated Electricity Cost per Month: Rp {cost_per_month:.2f}\n"
                f"Estimated Time to Check All Combinations: {total_years:.2f} years\n"
                f"Last checked address balance: {balance_status}")
        sys.stdout.write(info)
        sys.stdout.flush()
        if balance is not None and balance > 0:
            print(f"\nFound non-zero balance! Seed Phrase: {seed_phrase}, Address: {address}, Balance: {balance} BTC")
            break

print("\nCompleted checking seed phrases.")

Addresses checked per second: 0.76
Total tried: 1
Power Usage: 2.72 W
Power Usage (kWh): 0.000001
CPU Cores: 16
CPU Frequency: 3200.00 MHz
CPU: Intel64 Family 6 Model 151 Stepping 2, GenuineIntel
Estimated Electricity Cost per Day: Rp 0.03
Estimated Electricity Cost per Month: Rp 0.92
Estimated Time to Check All Combinations: 226707516429423601415915063738368.00 years
Addresses checked per second: 0.64C
Total tried: 2
Power Usage: 2.56 W
Power Usage (kWh): 0.000001
CPU Cores: 16
CPU Frequency: 3200.00 MHz
CPU: Intel64 Family 6 Model 151 Stepping 2, GenuineIntel
Estimated Electricity Cost per Day: Rp 0.03
Estimated Electricity Cost per Month: Rp 0.87
Estimated Time to Check All Combinations: 271477247755048523466510939717632.00 years
Addresses checked per second: 0.61C
Total tried: 3
Power Usage: 2.40 W
Power Usage (kWh): 0.000001
CPU Cores: 16
CPU Frequency: 3200.00 MHz
CPU: Intel64 Family 6 Model 151 Stepping 2, GenuineIntel
Estimated Electricity Cost per Day: Rp 0.03
Estimated Electr

KeyboardInterrupt: 

In [3]:
from mnemonic import Mnemonic
from bip_utils import Bip39SeedGenerator, Bip44, Bip44Coins, Bip44Changes
import requests
from bs4 import BeautifulSoup

# Fungsi untuk memeriksa saldo wallet menggunakan Arkham Intelligence
def check_balance(address):
    url = f"https://platform.arkhamintelligence.com/explorer/address/{address}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        balance_element = soup.select_one('#__next > div > div:nth-child(3) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > span:nth-child(1)')
        if balance_element:
            balance_text = balance_element.get_text().replace(',', '').strip()
            balance = float(balance_text)
            return balance
        else:
            print(f"Could not find balance element for address {address}")
            return None
    except (requests.RequestException, ValueError) as e:
        print(f"Error checking balance for address {address}: {e}")
        return None

# Fungsi untuk mendapatkan alamat Bitcoin dari seed phrase
def get_btc_address(seed_phrase):
    seed_bytes = Bip39SeedGenerator(seed_phrase).Generate()
    bip44_mst_ctx = Bip44.FromSeed(seed_bytes, Bip44Coins.BITCOIN)
    bip44_acc_ctx = bip44_mst_ctx.Purpose().Coin().Account(0)
    bip44_chg_ctx = bip44_acc_ctx.Change(Bip44Changes.CHAIN_EXT)
    bip44_addr_ctx = bip44_chg_ctx.AddressIndex(0)
    return bip44_addr_ctx.PublicKey().ToAddress()

# Fungsi untuk mendapatkan alamat dan saldo dari seed phrase
def get_address_and_balance(seed_phrase):
    address = get_btc_address(seed_phrase)
    balance = check_balance(address)
    return address, balance

# Contoh penggunaan fungsi
seed_phrase = "abandon ability able about above absent absorb abstract absurd abuse access actress"
address, balance = get_address_and_balance(seed_phrase)
print(f"Address: {address}\nBalance: {balance} BTC")


Could not find balance element for address 1LcStvg9StowHkWShpBq2YckG4xSycKXFT
Address: 1LcStvg9StowHkWShpBq2YckG4xSycKXFT
Balance: None BTC
